In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [37]:
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [24]:
def create_features(rel_df, pos):
    if pos == "rb":
        for col in rel_df.columns[4:]:
            if "FPTS" not in col:
                if "_avg" in col:
                    new_c = rel_df["FPTS/G_MISC_avg"]*rel_df[col]
                elif "_ewm" in col:
                    new_c = rel_df["FPTS/G_MISC_ewm"]*rel_df[col]
                
                new_name = col+"_FP"

                rel_df[new_name] = new_c
    return rel_df

In [64]:
def train(pos):
    df = pd.read_csv(f"processed data/{pos}_proc_data.csv")
    df = df[df["Rookie"] == 0]
    df = df.drop(["Rookie", "Career_Years"], axis=1)
    df_feats = create_features(df, pos)
    
    
    with open(f"{pos}_feats", "rb") as fp:   # Unpickling
        model_cols = pickle.load(fp)
        
    model_df = df_feats[model_cols]
    
    X = model_df.drop(["FPTS_TG", "Season"], axis=1)
    y = model_df[["FPTS_TG"]]

    lr = LinearRegression()
    
    sc = preprocessing.StandardScaler()
    X = sc.fit_transform(X)
    
    
    lr.fit(X, y)
    
    return lr, sc#[model_df.Season == 2022]

In [83]:
def predict_ns(pos, model, sc):
    df = pd.read_csv(f"processed data/{pos}_ns_proc.csv")
    df = df[df["Rookie"] == 0]
    df = df.drop(["Rookie", "Career_Years"], axis=1)
    df = df[~df["YDS_RUSHING/G_avg"].isna()]
    #return df
    df_feats = create_features(df, pos)
    
    
    with open(f"{pos}_feats", "rb") as fp:   # Unpickling
        model_cols = pickle.load(fp)
    #return model_cols
    model_df = df_feats[model_cols]
    
    X = model_df.drop(["FPTS_TG", "Season"], axis=1)
    X_sc = sc.transform(X)
    
    pred = model.predict(X_sc)
    
    df_feats["FPTS_TG_pred"] = pred
    
    df_board = df_feats[["pid", "Player", "Season", "AVG", "FPTS_TG_pred"]+model_cols[3:]]
    return df_board

In [66]:
model, scaler = train("rb")

In [85]:
pred_board = predict_ns("rb", model, scaler)

In [90]:
pred_board.sort_values("FPTS_TG_pred", ascending=False).head(20)

,pid,Player,Season,AVG,FPTS_TG_pred,FPTS/G_MISC_avg,YDS_RUSHING/G_avg,FPTS/G_MISC_ewm,YDS_RUSHING/G_ewm,TD_RUSHING/G_ewm,...,TD_RUSHING/G_avg_FP,REC_RECEIVING/G_avg_FP,TGT_RECEIVING/G_avg_FP,YDS_RECEIVING/G_avg_FP,ATT_RUSHING/G_ewm_FP,YDS_RUSHING/G_ewm_FP,TD_RUSHING/G_ewm_FP,REC_RECEIVING/G_ewm_FP,TGT_RECEIVING/G_ewm_FP,YDS_RECEIVING/G_ewm_FP
0,16393,Christian McCaffrey SF (9),2023,1.7,17.306169,21.800000,72.957589,20.425414,70.215253,0.678104,...,17.535986,126.452976,149.430375,1117.617341,323.281638,1434.175590,13.850548,112.742825,133.999399,999.290027
1,16483,Austin Ekeler LAC (5),2023,4.0,15.496135,17.075000,49.643382,17.630390,51.897749,0.551158,...,7.693166,93.140358,113.529917,790.873824,205.319159,914.977537,9.717123,97.221966,118.845876,779.094167
8,18269,Josh Jacobs LV (13),2023,21.7,14.621465,14.850000,78.707541,15.349271,80.103423,0.671905,...,9.817127,38.818303,48.664548,279.793710,275.202685,1229.529173,10.313258,43.801040,54.044266,313.992862
6,15514,Derrick Henry TEN (7),2023,17.0,14.232898,20.025000,110.651042,19.909307,108.794905,1.017221,...,20.984531,33.541875,43.053750,325.322812,467.466850,2166.031200,20.252164,36.321265,45.598461,364.687800
12,22982,Breece Hall NYJ (7),2023,30.3,13.144027,15.100000,66.142857,15.100000,66.142857,0.571429,...,8.628571,40.985714,66.871429,470.257143,172.571429,998.757143,8.628571,40.985714,66.871429,470.257143
16,16673,Aaron Jones GB (6),2023,39.3,13.095908,15.325000,66.453746,14.545178,65.212953,0.383179,...,7.766598,51.173615,66.351991,399.538472,187.356718,948.534007,5.573405,49.322822,62.778216,368.192374
4,17246,Nick Chubb CLE (5),2023,11.3,13.011955,15.475000,90.481530,15.514382,90.119872,0.697758,...,10.744722,25.534292,32.979875,218.052015,265.264150,1398.154108,10.825278,24.601827,31.941200,213.510957
7,18705,Tony Pollard DAL (7),2023,18.3,12.349412,8.975000,42.097917,10.261691,47.311896,0.323714,...,2.421380,17.473203,23.194766,145.507188,93.670084,485.500045,3.321852,22.059075,29.363973,190.862708
2,17240,Saquon Barkley NYG (13),2023,8.3,12.229782,12.050000,54.627747,12.319120,57.691774,0.347252,...,3.703554,40.641853,57.047150,312.104104,179.309353,710.711869,4.277836,41.121427,57.078015,295.007917
9,22726,Rhamondre Stevenson NE (11),2023,24.3,12.151837,10.800000,55.838235,11.108831,56.754132,0.344879,...,3.838235,28.217647,36.052941,189.079412,131.384728,630.472067,3.831204,31.780732,40.587521,208.318555
